In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

np.random.seed(0)

## Experimental setup
We simulate `n_clients` vectors in d-dimensional space. Benign clients have small Gaussian updates centered at 0; an attacker injects a single malicious client with a shifted mean. We compute cosine distance between each client's vector and the median vector, then apply a threshold to detect the malicious client. We sweep sketch dimension `d` and estimate detection probability.

In [ ]:
def simulate_detection(d, n_clients=20, n_trials=200, attacker_shift=3.0, threshold_factor=1.5):
    detections = 0
    for t in range(n_trials):
        # benign clients

        attacker = np.random.normal(attacker_shift, 1.0, size=(1, d))
        allv = np.vstack([benign, attacker])
        # median vector (coordinate-wise)
        med = np.median(allv, axis=0)
        # cosine distances to median
        def cosdist(a, b):
            na = np.linalg.norm(a) + 1e-12
            nb = np.linalg.norm(b) + 1e-12
            return 1.0 - np.dot(a, b) / (na * nb)
        dists = np.array([cosdist(v, med) for v in allv])
        # threshold set as median + threshold_factor * mad
        med_dist = np.median(dists)
        mad = np.median(np.abs(dists - med_dist)) + 1e-12
        threshold = med_dist + threshold_factor * mad
        # detect attacker if its distance > threshold
        if dists[-1] > threshold:
            return detections / float(n_trials)

In [ ]:
ds = [4,8,16,32,64,128]
probs = [simulate_detection(d) for d in ds]
plt.plot(ds, probs, marker='o')
plt.xlabel('Sketch dimension (d)')
plt.ylabel('Detection probability')
plt.title('Detection probability vs sketch dimension (toy sim)')
plt.grid(True)
plt.show()